# modeling4-2-nli-binaryclassifier-BinaryInferenceDataset
- CustomDataset Class for Binray input
- New CustomDatset following for  
    __modeling6-3-nli-binaryclassifier-datapreprocessing-checkingoutput-adding gold__  
    __Converting each data into binary dataset3__

In [27]:
from tqdm.auto import tqdm
import json
import os
import argparse
import transformers
import torch
import numpy as np
import random
from torch import nn
from torch.utils.data import Dataset
from transformers import (
    AutoModelForSequenceClassification, 
    AutoModel, 
    AutoConfig, 
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
)
from util.arguments import ModelArguments, DataTrainingArguments 
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

import json
from copy import deepcopy
from pprint import pprint
from util import utils
# from transformers.models.longformer impor LongformerForSequenceClassification, LongformerSequenceClassifierOutput
# from transformers.modeling_outputs import LongformerSequenceClassifierOutput
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
parser = HfArgumentParser(
        (ModelArguments, DataTrainingArguments, TrainingArguments)
    )
# parser.parse_args_into_dataclasses([])
# args = ["--foo", "1", "--baz", "quux", "--bar", "0.5"]
args = ["--model_name_or_path", 'roberta-large', '--output_dir', './']
# (example,) = parser.parse_args_into_dataclasses(args, look_for_args_file=False)
# args = parser.parse_args(args=[])
model_args, data_args, training_args = parser.parse_args_into_dataclasses(args)
# print(model_args.model_name_or_path)

In [3]:
print(f'model_args : \n ')
pprint(model_args)
print(f'data_args : \n ')
pprint(data_args)
print(f'training_args : \n ')
pprint(data_args)

model_args : 
 
ModelArguments(model_architecture='roberta-large', model_name_or_path='roberta-large', git_tag='v1.1', config_name=None, tokenizer_name=None, max_seq_length=200)
data_args : 
 
DataTrainingArguments(data='NQ-DEV-DPR/5-fold/1', train_file='/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1_partial.json', eval_file='/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1_partial.json', num_labels=2, overwrite_cache=False, pad_to_max_length=False, dataset_class='BinaryCustomDatasetShuffle')
training_args : 
 
DataTrainingArguments(data='NQ-DEV-DPR/5-fold/1', train_file='/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1_partial.json', eval_file='/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/binary_data/binary_ex_ctx100id_split_train_1_partial.json', num_labels=2, overwrite_cach

In [4]:
parser = argparse.ArgumentParser()
MY_DICT = {'key' : 'value'}

In [5]:
parser.add_argument('-s', '--seed', type=int, default=42, help='random seed (default : 42)')
parser.add_argument('-m', '--model', type=str, default='BaseModel', help='model type (default:BaseModel)')
parser.add_argument('--train-dataset', type=str, default='NQ', help='train dataset (default:Natural Question)')
parser.add_argument('--eval_dataset', type=str, default='NQ', help='eval dataset (default:Natural Question)')
parser.add_argument('--output_dir', type=str, default='output directory', help='output directory')
# parser.add_argument('--cache_directory', type = str, default = '', help='default transformer cahce directory')

_StoreAction(option_strings=['--output_dir'], dest='output_dir', nargs=None, const=None, default='output directory', type=<class 'str'>, choices=None, help='output directory', metavar=None)

In [6]:
args = parser.parse_args([])

In [7]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

In [8]:
# print(args)

In [9]:
seed_everything(args.seed)

In [10]:
# config file are updated when executing .save_pretrained('./')
# model architecture depends on model_name
model = AutoModelForSequenceClassification.from_pretrained('roberta-large', num_labels=2)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifie

In [11]:
# TEST1
# model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-large-4096')
# model = AutoModel.from_pretrained('allenai/longformer-large-4096')
# type(model)

# TEST2
# Custom LongformerForSequenceClassification
# class CustomSequenceClassification(nn.Module):
#     def __init__(self, model_name, num_labels):
#         super(CustomSequenceClassification, self).__init__()
#         self.num_labels = num_labels
#         self.model_name = model_name
        
#         # Base Model Configuration
#         self.model = AutoModel.from_pretrained(self.model_name, add_pooling_layer = False)
#         self.config = self.model.config
        
#         # Custom Layers for Sequence Classification
#         self.dense = nn.Linear(self.config.hidden_size, self.config.hidden_size)
#         self.dropout = nn.Dropout(self.config.hidden_dropout_prob)
#         self.out_proj = nn.Linear(self.config.hidden_size, self.num_labels)
        
#     def forward(self, 
#                 input_ids=None, 
#                 attention_mask=None, 
#                 labels=None):
        
#         outputs = self.model(
#             input_ids=input_ids, 
#             attention_mask=attention_mask,
#         )
        
#         # outputs[0] -> last hidden layer 
#         sequence_output = outputs[0]
        
#         # [batch, sequence, hidden] -> [:, 0, :] -> hidden state of <s> or [CLS] token 
#         hidden_states = hidden_states[:, 0, :]
#         hidden_states = self.dropout(hidden_states)
#         hidden_states = self.dense(hidden_states)
#         hidden_states = torch.tanh(hidden_states)
#         hidden_states = self.dropout(hidden_states)
#         logits = self.out_proj(hidden_states)
        
#         loss = None
#         if labels is not None:
#             # single_label_classification
#             loss_fct = CrossEntropyLoss()
#             loss = loss_fct(logits.view(-1, self.num_labels), lablels.view(-1))
            
#         return SequenceClassifierOutput(
#             loss=loss, 
#             logits=logits, 
#             hidden_states=outputs.hidden_states,
#             attentions=outputs.attentions,
#             global_attentions=outputs.gloabl_attentions
#         )

# TEST3
# class CustomSequenceClassification(AutoModelForSequenceClassification):
#     def __init__(self, model_name, num_labels):
#         super().__init__()
#         self.model_name = model_name
#         self.num_labels = num_labels
#         self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name, num_labels=self.num_labels)
        
#     def forward(self, 
#                 input_ids=None, 
#                 attention_mask=None, 
#                 labels=None):
        
#         outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        
#         return outputs

## nq

In [12]:
# nq_test_file = '/data/philhoon-relevance/KILT/kilt-dpr-retrieval/nq-dev-multikilt.json'
# nq_dpr_dev = open_json(nq_test_file)

In [13]:
# nq_train_file = '/data/philhoon-relevance/KILT/kilt-dpr-retrieval/nq-train-multikilt.json'
# nq_dpr_train = open_json(nq_train_file)

In [14]:
# print(len(nq_dpr_dev))
# print(len(nq_dpr_train))

# decisive_binary_gold

In [19]:
binary_train_file = '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR\
/5-fold/1/decisive_binary_gold_data/\
binary_decisive_gold_ctx100id_split_train_1.json'

binary_dev_file = '/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/\
5-fold/1/decisive_binary_gold_data/\
binary_decisive_gold_ctx100id_split_dev_1.json'

# binary_dev_file = 'binary_ex_ctx100id_split_dev_1.json'
max_length = 200

In [20]:
binary_train_file

'/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/decisive_binary_gold_data/binary_decisive_gold_ctx100id_split_train_1.json'

In [21]:
binary_dev_file

'/data/philhoon-relevance/binary-classification/NQ-DEV-DPR/5-fold/1/decisive_binary_gold_data/binary_decisive_gold_ctx100id_split_dev_1.json'

In [22]:
binary_train = utils.open_json(binary_train_file)
binary_dev = utils.open_json(binary_dev_file)

In [23]:
print(len(binary_train))
print(len(binary_dev))

6721
1704


In [24]:
model_name = 'roberta-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [25]:
type(binary_train)

list

In [26]:
binary_train[0]

{'id': 4,
 'question': 'what is the smallest prime number that is greater than 30',
 'ctx': {'id': '2760888',
  'title': 'Formula for primes',
  'text': 'of equations of degree only 4, but in 58 variables. The first such formula known was established by , who proved that there exists a real number "A" such that, if then is a prime number for all positive integers "n". If the Riemann hypothesis is true, then the smallest such "A" has a value of around 1.3063778838630806904686144926... and is known as Mills\' constant. This value gives rise to the primes formula_29, formula_30, formula_31, ... Very little is known about the constant "A" (not even whether it is rational). This formula has no practical value, because there is no known'},
 'em': '1',
 'answers': ['31'],
 'gold': '31',
 'inference': '31'}

In [28]:
# cnt = 0
# for idx, instance in tqdm(enumerate(binary_dev)):
#     input_ = 'question: ' + instance['question'] + ', '\
#         ' answer: ' + instance['gold'] + ', '\
#         ' title: ' + instance['ctx']['title'] + ', '\
#         ' context : ' + instance['ctx']['text']
    
#     output = tokenizer(
#             input_, 
#             # return_tensors="pt", will be applied later through collator
#             # padding=True, will be padded later through collate
#             truncation=True, 
#             add_special_tokens=True, 
#             max_length=max_length)
# #     print(input_)
# #     print(len(output['input_ids']))
#     if len(output['input_ids']) > max_length:
#         cnt += 1
        

In [29]:
class BinaryCustomDatasetDecisiveBinaryGold(torch.utils.data.Dataset):
    def __init__(self, instances, tokenizer, max_length, shuffle = False):
        if shuffle:
            random.shuffle(instances)
        self.instances = instances
        self.tokenizer = tokenizer
        self.sep_token = tokenizer.sep_token
        self.max_length = max_length

    def __len__(self):
        return len(self.instances)

    def __getitem__(self, idx):
        input_ = 'question: ' + self.instances[idx]['question'] + ', '\
            ' answer: ' + self.instances[idx]['gold'] + ', '\
            ' title: ' + self.instances[idx]['ctx']['title'] + ', '\
            ' context : ' + self.instances[idx]['ctx']['text']
        output = self.tokenizer(
            input_,
            # return_tensors="pt", will be applied later through collator
            # padding=True, will be padded later through collate
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_length)

        item = {key: val for key, val in output.items()}
        # item['labels'] = torch.tensor(int(self.instances[idx]['em']))
        item['labels'] = int(self.instances[idx]['em'])

        return item

In [30]:
train_decisive_binary_gold = BinaryCustomDatasetDecisiveBinaryGold(binary_train, tokenizer, max_length)
dev_decisive_binary_gold = BinaryCustomDatasetDecisiveBinaryGold(binary_dev, tokenizer, max_length)

In [39]:
i = 100

In [40]:
print(train_decisive_binary_gold[i])

{'input_ids': [0, 40018, 35, 147, 473, 24, 1067, 59, 5, 18023, 11, 5, 7255, 6, 1437, 1948, 35, 6776, 38, 6, 7162, 132, 6, 1437, 1270, 35, 315, 532, 20795, 6, 1437, 5377, 4832, 315, 532, 20795, 20, 315, 532, 20795, 16, 10, 5044, 13952, 18023, 17171, 30, 6776, 38, 6, 7162, 132, 9, 5, 315, 532, 5879, 6, 61, 982, 35, 22, 28588, 11649, 8, 2228, 34830, 5658, 28, 1553, 33938, 196, 566, 5, 484, 532, 1666, 309, 7, 49, 7091, 31415, 1666, 479, 20, 3031, 2271, 26229, 1258, 5658, 28, 156, 624, 130, 10426, 71, 5, 78, 529, 9, 5, 1148, 9, 5, 315, 532, 6, 8, 624, 358, 7757, 25569, 9, 2724, 10426, 72, 7162, 132, 9, 5, 501, 212, 8352, 982, 35, 22, 28588, 11649, 5658, 28, 1553, 33938, 196, 566, 5, 484, 532, 309, 7, 49, 7091, 1530, 6, 10581, 5, 1086, 346, 9, 5151, 11, 349, 331, 6, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [41]:
print(tokenizer.convert_ids_to_tokens(train_decisive_binary_gold[i]['input_ids']))

['<s>', 'question', ':', 'Ġwhere', 'Ġdoes', 'Ġit', 'Ġtalk', 'Ġabout', 'Ġthe', 'Ġcensus', 'Ġin', 'Ġthe', 'Ġconstitution', ',', 'Ġ', 'Ġanswer', ':', 'ĠArticle', 'ĠI', ',', 'ĠSection', 'Ġ2', ',', 'Ġ', 'Ġtitle', ':', 'ĠUnited', 'ĠStates', 'ĠCensus', ',', 'Ġ', 'Ġcontext', 'Ġ:', 'ĠUnited', 'ĠStates', 'ĠCensus', 'ĠThe', 'ĠUnited', 'ĠStates', 'ĠCensus', 'Ġis', 'Ġa', 'Ġdec', 'ennial', 'Ġcensus', 'Ġmandated', 'Ġby', 'ĠArticle', 'ĠI', ',', 'ĠSection', 'Ġ2', 'Ġof', 'Ġthe', 'ĠUnited', 'ĠStates', 'ĠConstitution', ',', 'Ġwhich', 'Ġstates', ':', 'Ġ"', 'Represent', 'atives', 'Ġand', 'Ġdirect', 'ĠTaxes', 'Ġshall', 'Ġbe', 'Ġapp', 'ortion', 'ed', 'Ġamong', 'Ġthe', 'Ġseveral', 'ĠStates', 'Ġ...', 'Ġaccording', 'Ġto', 'Ġtheir', 'Ġrespective', 'ĠNumbers', 'Ġ...', 'Ġ.', 'ĠThe', 'Ġactual', 'ĠEn', 'umer', 'ation', 'Ġshall', 'Ġbe', 'Ġmade', 'Ġwithin', 'Ġthree', 'ĠYears', 'Ġafter', 'Ġthe', 'Ġfirst', 'Ġmeeting', 'Ġof', 'Ġthe', 'ĠCongress', 'Ġof', 'Ġthe', 'ĠUnited', 'ĠStates', ',', 'Ġand', 'Ġwithin', 'Ġevery', 'Ġsub

In [42]:
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(train_decisive_binary_gold[i]['input_ids']))

'<s>question: where does it talk about the census in the constitution,  answer: Article I, Section 2,  title: United States Census,  context : United States Census The United States Census is a decennial census mandated by Article I, Section 2 of the United States Constitution, which states: "Representatives and direct Taxes shall be apportioned among the several States ... according to their respective Numbers ... . The actual Enumeration shall be made within three Years after the first meeting of the Congress of the United States, and within every subsequent Term of ten Years." Section 2 of the 14th Amendment states: "Representatives shall be apportioned among the several States according to their respective numbers, counting the whole number of persons in each State,</s>'

In [43]:
train_decisive_binary_gold[i]['labels']

1

In [ ]:
pprint(binary_train[0])

In [ ]:
def preprocess_function(examples):
        # Tokenize the texts
        texts = (
            (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
        )
        result = tokenizer(*texts, padding=padding, max_length=args.max_length, truncation=True)

        if "label" in examples:
            if label_to_id is not None:
                # Map labels to IDs (not necessary for GLUE tasks)
                result["labels"] = [label_to_id[l] for l in examples["label"]]
            else:
                # In all cases, rename the column to labels because the model will expect that.
                result["labels"] = examples["label"]
        return result

In [ ]:
class BinarySentenceDataset(torch.utils.data.Dataset):
    def __init__(self, instances, tokenizer, max_length, shuffle = False):
        if shuffle:
            random.shuffle(instances)
        self.instances = instances
        self.tokenizer = tokenizer
        self.sep_token = tokenizer.sep_token
        self.max_length = max_length
    
    def __len__(self):
        return len(self.instances)
    
    def __getitem__(self, idx):
        text1_ = 'question: ' + self.instances[idx]['question']
        
        text2_ = 'title: ' + self.instances[idx]['ctx']['title'] + \
                    ' context : ' + self.instances[idx]['ctx']['text']
        output = self.tokenizer(
            text1_, text2_, 
            # return_tensors="pt", will be applied later through collator
            # padding=True, will be padded later through collate
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_length)

        item = {key: val for key, val in output.items()}
        item['labels'] = int(self.instances[idx]['em'])

        return item

In [ ]:
train_dataset_testing = BinarySentenceDataset(binary_train,
    tokenizer,
    max_length,
    False)

In [ ]:
print(train_dataset_testing[0])

In [ ]:
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(train_dataset_testing[1]['input_ids']))

In [ ]:
tokenizer.cls_token

In [ ]:
class BinaryCustomDatasetShuffle(torch.utils.data.Dataset):
    def __init__(self, instances, tokenizer, max_length, shuffle = False):
        if shuffle:
            random.shuffle(instances)
        self.instances = instances
        self.tokenizer = tokenizer
        self.sep_token = tokenizer.sep_token
        self.max_length = max_length

    def __len__(self):
        return len(self.instances)

    def __getitem__(self, idx):
        input_ = 'question: ' + self.instances[idx]['question'] + \
                 ' title: ' + self.instances[idx]['ctx']['title'] + \
                 ' context : ' + self.instances[idx]['ctx']['text']
        output = self.tokenizer(
            input_,
            # return_tensors="pt", will be applied later through collator
            # padding=True, will be padded later through collate
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_length)

        item = {key: val for key, val in output.items()}
        # item['labels'] = torch.tensor(int(self.instances[idx]['em']))
        item['labels'] = int(self.instances[idx]['em'])

        return item

In [ ]:
train_dataset = BinaryCustomDatasetShuffle(binary_train,
    tokenizer,
    max_length,
    False)

In [ ]:
train_dataset.instances[0]['em']

In [ ]:
print(train_dataset[1])

In [ ]:
pprint(binary_train[0])

In [ ]:
binary_train[0]

In [ ]:
instances = binary_train
idx = 0
sep_token = tokenizer.sep_token
max_length = 200

In [ ]:
instances[0]

In [ ]:
input_ = 'question: ' + instances[idx]['question'] + \
        ' title: ' + instances[idx]['ctx']['title'] + \
        ' context : ' + instances[idx]['ctx']['text']

In [ ]:
output = tokenizer(
            input_, 
            # return_tensors="pt", will be applied later through collator
            # padding=True, will be padded later through collate
            truncation=True, 
            add_special_tokens=True, 
            max_length=max_length)

In [ ]:
input_

In [ ]:
print(len(output['input_ids']))
print(output['input_ids'])

In [ ]:
print(tokenizer.convert_ids_to_tokens(output['input_ids']))
print(len(tokenizer.convert_ids_to_tokens(output['input_ids'])))

In [ ]:
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(output['input_ids']))

In [ ]:
from tqdm import tqdm

In [ ]:
cnt = 0
for idx, instance in tqdm(enumerate(instances)):
    input_ = 'question: ' + instance['question'] + \
        ' title: ' + instance['ctx']['title'] + \
        ' context : ' + instance['ctx']['text']
    
    output = tokenizer(
            input_, 
            # return_tensors="pt", will be applied later through collator
            # padding=True, will be padded later through collate
            truncation=True, 
            add_special_tokens=True, 
            max_length=max_length)
    
    if len(output['input_ids']) > max_length:
        cnt += 1
print(cnt)

In [ ]:
binary_train[0]

In [ ]:
'question: ' + binary_train[0]['question'] + ' title: ' + binary_train[0]['ctx']['title'] + ' context : ' + binary_train[0]['ctx']['text']

In [ ]:
binary_train[0]['ctx']['title']

In [ ]:
binary_train[0]['ctx']['text']

In [ ]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

In [ ]:
data_collator = DataCollatorWithPadding(
        tokenizer, 
        pad_to_multiple_of=8,
    )

In [ ]:
# Trainer implement dataloader inside
# data_loader = DataLoader(
#     dataset, 
#     batch_size=8, 
#     shuffle=True, 
#     sampler=None,
#     batch_sampler=None, 
#     num_workers=1, 
#     collate_fn=data_collator,
#     pin_memory=False, 
#     drop_last=False, 
#     timeout=0,
#     worker_init_fn=None)

# Metric

In [ ]:
from datasets import load_metric

In [ ]:
import evaluate

In [ ]:
metric = evaluate.load("xnli")

In [ ]:
metric

In [ ]:
metric3 = evaluate.load("f1")

In [ ]:
metric3